In [22]:
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [2]:
#GET THE DATA
dataset = pd.read_csv("amazon_reviews3.tsv", sep = "\t")

In [3]:
dataset.columns

Index(['Unnamed: 0', 'eDOC_ID', 'LABEL', 'RATING', 'VERIFIED_PURCHASE',
       'PRODUCT_CATEGORY', 'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE',
       'REVIEW_TEXT', 'POS_COUNTS', 'NUM_NOUNS', 'NUM_VERBS', 'NUM_ADJECTIVES',
       'NUM_ADVERBS', 'REVIEW_LENGTH', 'SENTIMENT_SCORE', 'TITLE_LENGTH',
       'AVERAGE_RATING', 'RATING_DEVIATION', 'NUM_REVIEWS'],
      dtype='object')

In [4]:
features = ['VERIFIED_PURCHASE','REVIEW_LENGTH', 'RATING', 'SENTIMENT_SCORE',
           'TITLE_LENGTH', 'RATING_DEVIATION', 'NUM_REVIEWS',
           'NUM_NOUNS', 'NUM_VERBS', 'NUM_ADJECTIVES', 'NUM_ADVERBS']

X = dataset[features].values
Y = dataset['LABEL'].values

#LABEL ENCODING Y -> 0,1

le = LabelEncoder()
Y = le.fit_transform(Y)
Y

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


array([0, 0, 0, ..., 1, 1, 1])

In [5]:
#ENCODE CATEGORICAL DATA
#-> ALL NUMERICAL DATA                   

In [6]:
#SPLIT DATASET INTO TRAINING AND TESTING SET

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [7]:
#FEATURE SCALING - NECESSARY FOR DEEP LEARNING

sc = StandardScaler()
X_TRAIN = sc.fit_transform(X_TRAIN)
X_TEST = sc.transform(X_TEST)

In [8]:
#INITIALISING THE ANN

ann = Sequential()

2023-06-14 18:15:55.852809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
#ADDING HIDDEN LAYER
ann.add(Dense(units = 16, activation = 'relu'))
#ADDING SECOND HIDDEN LAYER
ann.add(Dense(units = 16, activation = 'relu'))
#ADDING OUTPUT LAYER
#->1: BINARY CLASSIFICATION; SIGMOID: PROBABILITY
ann.add(Dense(units=1, activation = 'sigmoid'))

In [10]:
#COMPILE THE MODEL
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [11]:
#FIT THE MODEL
ann.fit(X_TRAIN, Y_TRAIN, epochs = 350, verbose = 1, batch_size = 32)

Epoch 1/350
525/525 [==============================] - 1s 1ms/step - loss: 0.4987 - accuracy: 0.7921
Epoch 2/350
525/525 [==============================] - 1s 1ms/step - loss: 0.4449 - accuracy: 0.8140
Epoch 3/350
525/525 [==============================] - 1s 1ms/step - loss: 0.4391 - accuracy: 0.8180
Epoch 4/350
525/525 [==============================] - 1s 1ms/step - loss: 0.4346 - accuracy: 0.8188
Epoch 5/350
525/525 [==============================] - 1s 1ms/step - loss: 0.4305 - accuracy: 0.8215
Epoch 6/350
525/525 [==============================] - 1s 1ms/step - loss: 0.4280 - accuracy: 0.8222
Epoch 7/350
393/525 [=====================>........] - ETA: 0s - loss: 0.4278 - accuracy: 0.8218

KeyboardInterrupt: 

In [ ]:
#PREDICT
predictions = ann.predict(X_TEST.tolist())

In [ ]:
predictions = predictions>0.5

In [ ]:
#CLASSIFICATION REPORT

print(classification_report(Y_TEST, predictions))
print(accuracy_score(Y_TEST, predictions))

In [ ]:
#INCORPORATE THE REVIEW_TEXT INTO TRAINING


In [ ]:
#REMOVE STOP WORDS
nlp = spacy.load('en_core_web_sm')
stop_words = nlp.Defaults.stop_words



def remove_stopwords(text):
    words = text.lower().split()
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)


In [ ]:
df = dataset
import string

In [ ]:
df['REVIEW_TEXT'] = df['REVIEW_TEXT'].apply(remove_stopwords)

In [ ]:
#REMOVE PUNCTUATION
def remove_punctuation(text):
    return ''.join([w for w in text if w not in string.punctuation])

In [ ]:
df['REVIEW_TEXT'] = df['REVIEW_TEXT'].apply(remove_punctuation)

In [ ]:
#STEMMING:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [ ]:
df['REVIEW_TEXT'] = df['REVIEW_TEXT'].apply(lambda d: stem_text(d))

In [ ]:
#LEMMATIZATION
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

In [ ]:
df['REVIEW_TEXT'][:1000] = df['REVIEW_TEXT'][:1000].apply(lambda d: lemmatize_text(d))

In [ ]:
df['REVIEW_TEXT'][1000:5000] = df['REVIEW_TEXT'][1000:5000].apply(lambda d: lemmatize_text(d))

In [ ]:
df['REVIEW_TEXT'][5000:10000] = df['REVIEW_TEXT'][5000:10000].apply(lambda d: lemmatize_text(d))

In [ ]:
df['REVIEW_TEXT'][10000:15000] = df['REVIEW_TEXT'][10000:15000].apply(lambda d: lemmatize_text(d))

In [ ]:
df['REVIEW_TEXT'][15000:21000] = df['REVIEW_TEXT'][15000:21000].apply(lambda d: lemmatize_text(d))

In [ ]:
df.to_csv('preprocessed_amazon_reviews.tsv', sep = '\t')

In [30]:
df = pd.read_csv('preprocessed_amazon_reviews.tsv', sep = '\t')

In [31]:
features_text = df['REVIEW_TEXT'].values
features_numeric = df[['REVIEW_LENGTH','TITLE_LENGTH',
                      'NUM_NOUNS', 'NUM_VERBS','NUM_ADVERBS', 
                      'NUM_ADJECTIVES','TITLE_LENGTH','SENTIMENT_SCORE', 
                      'VERIFIED_PURCHASE', 'RATING','RATING_DEVIATION']].values

In [32]:
labels = df['LABEL'].values
#LABEL ENCODING Y -> 0,1

le = LabelEncoder()
labels = le.fit_transform(labels)

In [33]:
#TRAIN-TEST SPLIT
X_text_train, X_text_test, X_numeric_train, X_numeric_test, y_train, y_test = train_test_split(
    features_text, features_numeric, labels, test_size=0.2, random_state=42
)

In [34]:
tfidf = TfidfVectorizer()
X_text_train_tfidf = tfidf.fit_transform(X_text_train)
X_text_test_tfidf = tfidf.transform(X_text_test)

In [35]:
# Standardize the numeric features
scaler = StandardScaler()
X_numeric_train = scaler.fit_transform(X_numeric_train)
X_numeric_test = scaler.transform(X_numeric_test)

In [36]:
# Concatenate the TF-IDF matrix and numeric features
X_train = np.hstack((X_text_train_tfidf.toarray(), X_numeric_train))
X_test = np.hstack((X_text_test_tfidf.toarray(), X_numeric_test))

In [37]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [47]:
model.fit(X_train, y_train, verbose=1, epochs=400, batch_size=32)

Epoch 1/400
525/525 [==============================] - 3s 5ms/step - loss: 9.1223e-05 - accuracy: 1.0000
Epoch 2/400
525/525 [==============================] - 3s 5ms/step - loss: 6.8052e-05 - accuracy: 1.0000
Epoch 3/400
525/525 [==============================] - 3s 5ms/step - loss: 5.0691e-05 - accuracy: 1.0000
Epoch 4/400
525/525 [==============================] - 3s 5ms/step - loss: 3.6872e-05 - accuracy: 1.0000
Epoch 5/400
525/525 [==============================] - 3s 6ms/step - loss: 2.8145e-05 - accuracy: 1.0000
Epoch 6/400
525/525 [==============================] - 3s 6ms/step - loss: 1.9956e-05 - accuracy: 1.0000
Epoch 7/400
525/525 [==============================] - 3s 5ms/step - loss: 1.4749e-05 - accuracy: 1.0000
Epoch 8/400
525/525 [==============================] - 3s 5ms/step - loss: 1.1132e-05 - accuracy: 1.0000
Epoch 9/400
525/525 [==============================] - 3s 5ms/step - loss: 7.9085e-06 - accuracy: 1.0000
Epoch 10/400
525/525 [==============================] -

525/525 [==============================] - 3s 5ms/step - loss: 3.1224e-08 - accuracy: 1.0000
Epoch 79/400
525/525 [==============================] - 3s 5ms/step - loss: 2.6950e-08 - accuracy: 1.0000
Epoch 80/400
525/525 [==============================] - 3s 5ms/step - loss: 2.2985e-08 - accuracy: 1.0000
Epoch 81/400
525/525 [==============================] - 3s 5ms/step - loss: 1.9930e-08 - accuracy: 1.0000
Epoch 82/400
525/525 [==============================] - 3s 5ms/step - loss: 1.7227e-08 - accuracy: 1.0000
Epoch 83/400
525/525 [==============================] - 3s 5ms/step - loss: 1.4731e-08 - accuracy: 1.0000
Epoch 84/400
525/525 [==============================] - 3s 5ms/step - loss: 1.2713e-08 - accuracy: 1.0000
Epoch 85/400
525/525 [==============================] - 3s 5ms/step - loss: 1.0949e-08 - accuracy: 1.0000
Epoch 86/400
525/525 [==============================] - 3s 5ms/step - loss: 9.4205e-09 - accuracy: 1.0000
Epoch 87/400
525/525 [==============================] - 3s 

Epoch 155/400
525/525 [==============================] - 3s 5ms/step - loss: 2.0106e-09 - accuracy: 1.0000
Epoch 156/400
525/525 [==============================] - 3s 5ms/step - loss: 1.8880e-09 - accuracy: 1.0000
Epoch 157/400
525/525 [==============================] - 3s 5ms/step - loss: 1.7643e-09 - accuracy: 1.0000
Epoch 158/400
525/525 [==============================] - 3s 5ms/step - loss: 1.6898e-09 - accuracy: 1.0000
Epoch 159/400
525/525 [==============================] - 3s 5ms/step - loss: 1.6639e-09 - accuracy: 1.0000
Epoch 160/400
525/525 [==============================] - 3s 5ms/step - loss: 0.0133 - accuracy: 0.9971
Epoch 161/400
525/525 [==============================] - 3s 5ms/step - loss: 0.0057 - accuracy: 0.9987
Epoch 162/400
525/525 [==============================] - 3s 5ms/step - loss: 0.0014 - accuracy: 0.9995
Epoch 163/400
525/525 [==============================] - 3s 5ms/step - loss: 2.3367e-05 - accuracy: 1.0000
Epoch 164/400
525/525 [==========================

525/525 [==============================] - 3s 5ms/step - loss: 2.2185e-07 - accuracy: 1.0000
Epoch 232/400
525/525 [==============================] - 3s 5ms/step - loss: 1.8900e-07 - accuracy: 1.0000
Epoch 233/400
525/525 [==============================] - 3s 5ms/step - loss: 1.5970e-07 - accuracy: 1.0000
Epoch 234/400
525/525 [==============================] - 3s 5ms/step - loss: 1.3745e-07 - accuracy: 1.0000
Epoch 235/400
525/525 [==============================] - 3s 5ms/step - loss: 1.1743e-07 - accuracy: 1.0000
Epoch 236/400
525/525 [==============================] - 3s 5ms/step - loss: 9.9595e-08 - accuracy: 1.0000
Epoch 237/400
525/525 [==============================] - 3s 5ms/step - loss: 8.5051e-08 - accuracy: 1.0000
Epoch 238/400
525/525 [==============================] - 3s 5ms/step - loss: 7.2371e-08 - accuracy: 1.0000
Epoch 239/400
525/525 [==============================] - 3s 5ms/step - loss: 6.1363e-08 - accuracy: 1.0000
Epoch 240/400
525/525 [============================

525/525 [==============================] - 3s 5ms/step - loss: 1.7168e-08 - accuracy: 1.0000
Epoch 308/400
525/525 [==============================] - 3s 5ms/step - loss: 1.5086e-08 - accuracy: 1.0000
Epoch 309/400
525/525 [==============================] - 3s 5ms/step - loss: 1.3277e-08 - accuracy: 1.0000
Epoch 310/400
525/525 [==============================] - 3s 5ms/step - loss: 1.1745e-08 - accuracy: 1.0000
Epoch 311/400
525/525 [==============================] - 3s 5ms/step - loss: 1.0472e-08 - accuracy: 1.0000
Epoch 312/400
525/525 [==============================] - 3s 5ms/step - loss: 9.3961e-09 - accuracy: 1.0000
Epoch 313/400
525/525 [==============================] - 3s 5ms/step - loss: 8.4506e-09 - accuracy: 1.0000
Epoch 314/400
525/525 [==============================] - 3s 5ms/step - loss: 7.6061e-09 - accuracy: 1.0000
Epoch 315/400
525/525 [==============================] - 3s 5ms/step - loss: 6.9130e-09 - accuracy: 1.0000
Epoch 316/400
525/525 [============================

525/525 [==============================] - 4s 7ms/step - loss: 7.6854e-10 - accuracy: 1.0000
Epoch 384/400
525/525 [==============================] - 4s 8ms/step - loss: 7.5116e-10 - accuracy: 1.0000
Epoch 385/400
525/525 [==============================] - 3s 6ms/step - loss: 7.3357e-10 - accuracy: 1.0000
Epoch 386/400
525/525 [==============================] - 3s 6ms/step - loss: 7.3403e-10 - accuracy: 1.0000
Epoch 387/400
525/525 [==============================] - 3s 6ms/step - loss: 7.4410e-10 - accuracy: 1.0000
Epoch 388/400
525/525 [==============================] - 3s 6ms/step - loss: 7.5036e-10 - accuracy: 1.0000
Epoch 389/400
525/525 [==============================] - 3s 7ms/step - loss: 7.6657e-10 - accuracy: 1.0000
Epoch 390/400
525/525 [==============================] - 4s 7ms/step - loss: 7.1372e-10 - accuracy: 1.0000
Epoch 391/400
525/525 [==============================] - 3s 6ms/step - loss: 7.2136e-10 - accuracy: 1.0000
Epoch 392/400
525/525 [============================

In [48]:
predictions = model.predict(X_test)
predictions = predictions>0.5

132/132 [==============================] - 0s 2ms/step


In [50]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.74      0.78      0.76      2115
           1       0.76      0.73      0.74      2085

   micro avg       0.75      0.75      0.75      4200
   macro avg       0.75      0.75      0.75      4200
weighted avg       0.75      0.75      0.75      4200

